In [1]:
require 'nn'
require 'hdf5'

### Loading the data:

In [76]:
myFile = hdf5.open('6-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [92]:
data

{
  valid_output : LongTensor - size: 3370x50
  train_1000 : LongTensor - size: 696825x7
  test : LongTensor - size: 3761x55
  nwords : LongTensor - size: 1
  train : LongTensor - size: 772670x7
  valid : LongTensor - size: 3370x55
}


### Creating the training set:

We need to remove the counts (used for the Count based language model)

In [81]:
nwords = 10001
train = data['train']:narrow(2,1,6)

In [82]:
train_input = train:narrow(2,1,5)
train_output = train:narrow(2,6,1)

In [89]:
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100
nnlm = nn.Sequential()

tanh = nn.Sequential()
tanh:add(nn.LookupTable(nwords,hid1/dwin))
tanh:add(nn.View(1,-1,hid1))
tanh:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
tanh:add(nn.Linear(hid1,hid2))
tanh:add(nn.Tanh())

nnlm:add(tanh)
nnlm:add(nn.Linear(hid2, nwords))
nnlm:add(nn.LogSoftMax())

In [84]:
nnlm:forward(train_input:narrow(1,1,1)):size()

 10001
[torch.LongStorage of size 1]



In [90]:
criterion = nn.ClassNLLCriterion()

In [86]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train_input[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [91]:
--with regularisation:

sq_mat = torch.zeros(10001,6)
norm_mat = torch.zeros(10001,6)

for i = 1, 20 do
    
    timer = torch.Timer()
    if i ~= 1 then
        sq_mat:copy(torch.pow(nnlm:findModules('nn.LookupTable')[1].weight,2))
        norm_mat:copy(torch.expandAs(sq_mat:sum(2), sq_mat))
        nnlm:findModules('nn.LookupTable')[1].weight:cdiv(norm_mat)
    end
    
    trainer = nn.StochasticGradient(nnlm, criterion)
    trainer.learningRate = 0.01
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
end


# StochasticGradient: training	


# current error = 6.4245431634655	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.4245431634655	
Epoch 1: 1867.2106969357	


# StochasticGradient: training	


# current error = 5.9158270506484	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.9158270506484	


Epoch 2: 1851.8256881237	


...

# current error = 5.1854116802198	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.1854116802198	
Epoch 18: 1804.7857210636	


# StochasticGradient: training	


# current error = 5.1802477525104	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.1802477525104	
Epoch 19: 1804.3937339783	


# StochasticGradient: training	


# current error = 5.163963303637	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 5.163963303637	
Epoch 20: 1804.8741080761	


In [137]:
torch.save('nnlm1',nnlm)

In [75]:
-- Without regularisation:

trainer = nn.StochasticGradient(nnlm, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 5
trainer:train(dataset)

# StochasticGradient: training	


# current error = 4.3886357784844	


# current error = 4.1514940863395	


# current error = 4.0717284582465	


# current error = 4.0314867838883	


# current error = 4.00779912249	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 4.00779912249	


### Tentative de Validation:

In [93]:
kag = data['valid_output']

In [95]:
valid = data['valid']

In [100]:
valid_topredict = valid:narrow(2,1,50)

In [101]:
valid_input = valid:narrow(2,51,5)

In [102]:
preds_valid = nnlm:forward(valid_input)

In [110]:
kag_pred_valid = torch.Tensor(preds_valid:size(1),50)
for i = 1,preds_valid:size(1) do
    kag_pred_valid[i]:copy(preds_valid[i]:index(1,valid_topredict[i])):exp()
end

In [114]:
norm_mat_ = torch.zeros(preds_valid:size(1),50)
norm_mat_:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat_)

In [135]:
acc = 0
for i = 1,preds_valid:size(1) do
    m,a = kag_pred_valid[i]:max(1)
    mm,aa = kag[i]:max(1)
    if aa[1] == a[1] then
        acc = acc + 1
    end
end
print(acc/3370.)

0.6053412462908	


In [138]:
function perplexity(distribution)
    local perp = torch.DoubleTensor(distribution:size(1), distribution:size(2))
    perp:copy(distribution)
    n = distribution:size(1) * distribution:size(2)
    return math.exp(-(1/n)* perp:log():sum())
end

In [149]:
perp = 0
for i = 1,preds_valid:size(1) do
    mm,aa = kag[i]:max(1)
    perp = perp + math.log(kag_pred_valid[i][aa[1]])
end


In [155]:
kag_pred_valid[preds_valid:size(1)][aa[1]]

0.99988141829309	


In [154]:
i

### Deep:

In [ ]:
N = 5
dwin = N-1
hid1 = 50
hid2 = 50
nnlm = nn.Sequential()

LT = nn.Sequential()
LT:add(nn.LookupTable(nwords,hid1))
LT:add(nn.View(1,-1,dwin*hid1))
LT:add(nn.Squeeze())

parall = nn.Parallel(2,1)
